# Task 1: Conversation Management & Summarization

## Objective
This notebook demonstrates conversation history management with the following features:
- Maintain running conversation history as a list of message dictionaries
- Implement truncation options by number of turns or word/character count
- Periodic summarization using Groq API to replace older history
- Demonstrate different truncation settings and summarization intervals

## Requirements
- Standard Python libraries only
- `requests` library for API calls
- `openai` client (Groq API is OpenAI-compatible)
- Groq API key set as environment variable

## Implementation Overview
1. **ConversationManager Class**: Handles conversation state and truncation
2. **Summarization Module**: Uses Groq API to summarize conversation history
3. **Demonstration**: Multiple conversation samples with different settings

## Setup and Dependencies

In [1]:
!pip install openai requests

In [2]:
import os
import json
from typing import List, Dict, Optional, Union
from openai import OpenAI
import requests
from datetime import datetime

## API Key Configuration

**Important**: Set your Groq API key, use the secrets manager in colab.


In [21]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [22]:
if not GROQ_API_KEY:
    raise ValueError("Please set your GROQ_API_KEY in the secrets")

client = OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1"
)
print("✅ Groq API client configured successfully")

✅ Groq API client configured successfully


## ConversationManager Class Implementation

In [32]:
class ConversationManager:
    def __init__(self, max_turns: Optional[int] = None, max_words: Optional[int] = None,
                 max_chars: Optional[int] = None, summarize_every: int = 5):
        self.conversation_history: List[Dict[str, str]] = []
        self.max_turns = max_turns
        self.max_words = max_words
        self.max_chars = max_chars
        self.summarize_every = summarize_every
        self.turn_count = 0
        self.summary_history: List[str] = []

    def add_message(self, role: str, content: str):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)
        self.turn_count += 1
        self._apply_truncation()
        if self.turn_count % self.summarize_every == 0:
            self._periodic_summarization()

    def _apply_truncation(self):
        # Truncate by number of turns
        if self.max_turns and len(self.conversation_history) > self.max_turns:
            self.conversation_history = self.conversation_history[-self.max_turns:]

        # Truncate by word count
        if self.max_words:
            self._truncate_by_words()

        # Truncate by character count
        if self.max_chars:
            self._truncate_by_chars()

    def _truncate_by_words(self):
        total_words = 0
        truncated_history = []

        for message in reversed(self.conversation_history):
            words_in_message = len(message["content"].split())
            if total_words + words_in_message <= self.max_words:
                truncated_history.insert(0, message)
                total_words += words_in_message
            else:
                break

        self.conversation_history = truncated_history

    def _truncate_by_chars(self):
        total_chars = 0
        truncated_history = []

        for message in reversed(self.conversation_history):
            chars_in_message = len(message["content"])
            if total_chars + chars_in_message <= self.max_chars:
                truncated_history.insert(0, message)
                total_chars += chars_in_message
            else:
                break

        self.conversation_history = truncated_history

    def _periodic_summarization(self):
        if len(self.conversation_history) < 3:  # Need some history to summarize
            return

        print(f"\n🔄 Performing summarization at turn {self.turn_count}...")

        conversation_text = "\n".join([
            f"{msg['role']}: {msg['content']}" for msg in self.conversation_history[:-2]
        ])

        summary = self._get_summary(conversation_text)

        if summary:
            # Replace older messages with summary
            self.summary_history.append(summary)
            # Keep last 2 messages and add summary as system message
            recent_messages = self.conversation_history[-2:]
            self.conversation_history = [
                {"role": "system", "content": f"Previous conversation summary: {summary}"}
            ] + recent_messages

            print(f"✅ Summarization complete. History compressed.")

    def _get_summary(self, conversation_text: str) -> Optional[str]:
        try:
            response = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that summarizes conversations. Provide a concise but comprehensive summary that captures the key points, topics discussed, and any important context."
                    },
                    {
                        "role": "user",
                        "content": f"Please summarize this conversation:\n\n{conversation_text}"
                    }
                ],
                max_tokens=500,
                temperature=0.3
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"❌ Error getting summary: {e}")
            return None

    def get_stats(self) -> Dict[str, Union[int, str]]:
        """Get conversation statistics."""
        total_words = sum(len(msg["content"].split()) for msg in self.conversation_history)
        total_chars = sum(len(msg["content"]) for msg in self.conversation_history)

        return {
            "total_turns": self.turn_count,
            "current_messages": len(self.conversation_history),
            "total_words": total_words,
            "total_chars": total_chars,
            "summaries_created": len(self.summary_history)
        }

    def display_conversation(self):
        """Display current conversation history in a readable format."""
        print("\n" + "="*50)
        print("📋 CURRENT CONVERSATION HISTORY")
        print("="*50)

        for i, message in enumerate(self.conversation_history, 1):
            role_emoji = {
                "user": "👤",
                "assistant": "🤖",
                "system": "⚙️"
            }
            emoji = role_emoji.get(message["role"], "❓")
            print(f"{i}. {emoji} {message['role'].upper()}: {message['content'][:100]}{'...' if len(message['content']) > 100 else ''}")

        stats = self.get_stats()
        print("\n" + "-"*50)
        print("📊 STATISTICS")
        for key, value in stats.items():
            print(f"  {key}: {value}")
        print("="*50)

print("✅ ConversationManager class defined successfully")

✅ ConversationManager class defined successfully


## Demonstration 1: Basic Conversation Management

In [27]:
manager1 = ConversationManager(max_turns=5, summarize_every=4)
print("🚀 Starting Demo 1: Basic Conversation Management")
print("Settings: max_turns=5, summarize_every=4\n")

# Sample conversation
conversation_samples = [
    ("user", "Hi, I'm looking for help with Python programming."),
    ("assistant", "Hello! I'd be happy to help you with Python programming. What specific topic would you like to learn about?"),
    ("user", "I want to learn about data structures, specifically lists and dictionaries."),
    ("assistant", "Great choice! Lists and dictionaries are fundamental data structures in Python. Lists are ordered collections that can store multiple items, while dictionaries store key-value pairs. Would you like me to show you some examples?"),
    ("user", "Yes, please show me some examples of list operations."),
    ("assistant", "Here are some common list operations: creating lists with [1, 2, 3], appending items with list.append(), accessing elements with indexing like list[0], and iterating with for loops."),
    ("user", "That's helpful. Now can you explain dictionary operations?"),
    ("assistant", "Certainly! Dictionaries use curly braces like {'key': 'value'}. You can access values with dict['key'], add new pairs with dict['new_key'] = 'new_value', and iterate through keys, values, or items.")
]

# Add messages one by one and observe behavior
for i, (role, content) in enumerate(conversation_samples, 1):
    print(f"\n➡️ Adding message {i}: {role} - {content[:50]}...")
    manager1.add_message(role, content)

    if i % 2 == 0:  # Display every 2 messages
        manager1.display_conversation()

🚀 Starting Demo 1: Basic Conversation Management
Settings: max_turns=5, summarize_every=4


➡️ Adding message 1: user - Hi, I'm looking for help with Python programming....

➡️ Adding message 2: assistant - Hello! I'd be happy to help you with Python progra...

📋 CURRENT CONVERSATION HISTORY
1. 👤 USER: Hi, I'm looking for help with Python programming.
2. 🤖 ASSISTANT: Hello! I'd be happy to help you with Python programming. What specific topic would you like to learn...

--------------------------------------------------
📊 STATISTICS
  total_turns: 2
  current_messages: 2
  total_words: 27
  total_chars: 156
  summaries_created: 0

➡️ Adding message 3: user - I want to learn about data structures, specificall...

➡️ Adding message 4: assistant - Great choice! Lists and dictionaries are fundament...

🔄 Performing summarization at turn 4...
✅ Summarization complete. History compressed.

📋 CURRENT CONVERSATION HISTORY
1. ⚙️ SYSTEM: Previous conversation summary: Here is a summary of the co

## Demonstration 2: Word Count Truncation

In [28]:
manager2 = ConversationManager(max_words=50, summarize_every=6)
print("🚀 Starting Demo 2: Word Count Truncation")
print("Settings: max_words=50, summarize_every=6\n")
long_conversation = [
    ("user", "I'm working on a complex data science project that involves machine learning, data preprocessing, feature engineering, model selection, hyperparameter tuning, and performance evaluation."),
    ("assistant", "That sounds like a comprehensive data science workflow! Let me help you break this down into manageable steps. First, let's talk about data preprocessing which includes cleaning, handling missing values, and data transformation."),
    ("user", "I have a dataset with missing values, outliers, and categorical variables that need encoding. What's the best approach?"),
    ("assistant", "For missing values, you can use imputation techniques like mean/median for numerical data or mode for categorical. For outliers, consider IQR method or Z-score. For categorical encoding, use one-hot encoding for nominal or label encoding for ordinal variables."),
    ("user", "What about feature selection and dimensionality reduction techniques?"),
    ("assistant", "Feature selection includes filter methods like correlation analysis, wrapper methods like recursive feature elimination, and embedded methods like LASSO. For dimensionality reduction, consider PCA for linear reduction or t-SNE for non-linear visualization.")
]

for i, (role, content) in enumerate(long_conversation, 1):
    print(f"\n➡️ Adding message {i} (length: {len(content.split())} words)")
    manager2.add_message(role, content)
    manager2.display_conversation()

🚀 Starting Demo 2: Word Count Truncation
Settings: max_words=50, summarize_every=6


➡️ Adding message 1 (length: 23 words)

📋 CURRENT CONVERSATION HISTORY
1. 👤 USER: I'm working on a complex data science project that involves machine learning, data preprocessing, fe...

--------------------------------------------------
📊 STATISTICS
  total_turns: 1
  current_messages: 1
  total_words: 23
  total_chars: 186
  summaries_created: 0

➡️ Adding message 2 (length: 33 words)

📋 CURRENT CONVERSATION HISTORY
1. 🤖 ASSISTANT: That sounds like a comprehensive data science workflow! Let me help you break this down into managea...

--------------------------------------------------
📊 STATISTICS
  total_turns: 2
  current_messages: 1
  total_words: 33
  total_chars: 228
  summaries_created: 0

➡️ Adding message 3 (length: 18 words)

📋 CURRENT CONVERSATION HISTORY
1. 👤 USER: I have a dataset with missing values, outliers, and categorical variables that need encoding. What's...

---------------------

## Demonstration 3: Character Count Truncation

In [29]:
manager3 = ConversationManager(max_chars=200, summarize_every=3)

print("🚀 Starting Demo 3: Character Count Truncation")
print("Settings: max_chars=200, summarize_every=3\n")

char_test_conversation = [
    ("user", "Hi there!"),
    ("assistant", "Hello! How can I help you today?"),
    ("user", "I need help with understanding object-oriented programming concepts in Python."),
    ("assistant", "OOP in Python includes classes, objects, inheritance, encapsulation, and polymorphism."),
    ("user", "Can you explain classes and objects with a simple example?"),
    ("assistant", "A class is a blueprint. For example: class Car: def __init__(self, brand): self.brand = brand. An object is an instance: my_car = Car('Toyota').")
]

for i, (role, content) in enumerate(char_test_conversation, 1):
    print(f"\n➡️ Adding message {i} (length: {len(content)} chars)")
    manager3.add_message(role, content)

    if i % 2 == 0:
        manager3.display_conversation()

🚀 Starting Demo 3: Character Count Truncation
Settings: max_chars=200, summarize_every=3


➡️ Adding message 1 (length: 9 chars)

➡️ Adding message 2 (length: 32 chars)

📋 CURRENT CONVERSATION HISTORY
1. 👤 USER: Hi there!
2. 🤖 ASSISTANT: Hello! How can I help you today?

--------------------------------------------------
📊 STATISTICS
  total_turns: 2
  current_messages: 2
  total_words: 9
  total_chars: 41
  summaries_created: 0

➡️ Adding message 3 (length: 78 chars)

🔄 Performing summarization at turn 3...
✅ Summarization complete. History compressed.

➡️ Adding message 4 (length: 86 chars)

📋 CURRENT CONVERSATION HISTORY
1. 🤖 ASSISTANT: Hello! How can I help you today?
2. 👤 USER: I need help with understanding object-oriented programming concepts in Python.
3. 🤖 ASSISTANT: OOP in Python includes classes, objects, inheritance, encapsulation, and polymorphism.

--------------------------------------------------
📊 STATISTICS
  total_turns: 4
  current_messages: 3
  total_words: 27
  to

## Demonstration 4: Periodic Summarization

In [30]:
manager4 = ConversationManager(summarize_every=3)

print("🚀 Starting Demo 4: Periodic Summarization")
print("Settings: summarize_every=3 (no other limits)\n")

summarization_conversation = [
    ("user", "I want to learn about web development with Python."),
    ("assistant", "Great! Python offers several frameworks for web development. The most popular ones are Django and Flask. Django is a full-featured framework, while Flask is more lightweight."),
    ("user", "What's the difference between Django and Flask?"),
    ("assistant", "Django is a 'batteries-included' framework with built-in features like an admin panel, ORM, and authentication. Flask is minimalist and gives you more control over components you use."),
    ("user", "Which one should I choose for a beginner project?"),
    ("assistant", "For beginners, I'd recommend starting with Flask. It's simpler to understand, has fewer abstractions, and helps you learn web development fundamentals better."),
    ("user", "Can you show me a simple Flask application example?"),
    ("assistant", "Here's a basic Flask app: from flask import Flask; app = Flask(__name__); @app.route('/') def home(): return 'Hello World!'; if __name__ == '__main__': app.run(debug=True)"),
    ("user", "That looks simple! How do I handle forms and user input in Flask?")
]

for i, (role, content) in enumerate(summarization_conversation, 1):
    print(f"\n➡️ Adding message {i}")
    manager4.add_message(role, content)

    if i in [3, 6, 9]:  # Show state at summarization points
        manager4.display_conversation()

🚀 Starting Demo 4: Periodic Summarization
Settings: summarize_every=3 (no other limits)


➡️ Adding message 1

➡️ Adding message 2

➡️ Adding message 3

🔄 Performing summarization at turn 3...
✅ Summarization complete. History compressed.

📋 CURRENT CONVERSATION HISTORY
1. ⚙️ SYSTEM: Previous conversation summary: This conversation has just begun. The user has expressed interest in ...
2. 🤖 ASSISTANT: Great! Python offers several frameworks for web development. The most popular ones are Django and Fl...
3. 👤 USER: What's the difference between Django and Flask?

--------------------------------------------------
📊 STATISTICS
  total_turns: 3
  current_messages: 3
  total_words: 84
  total_chars: 545
  summaries_created: 1

➡️ Adding message 4

➡️ Adding message 5

➡️ Adding message 6

🔄 Performing summarization at turn 6...
✅ Summarization complete. History compressed.

📋 CURRENT CONVERSATION HISTORY
1. ⚙️ SYSTEM: Previous conversation summary: Here is a summary of the conversation:

*

## Utility Functions for Analysis

In [31]:
def compare_managers(managers: List[ConversationManager], labels: List[str]):
    print("\n" + "="*70)
    print("📊 COMPARISON OF CONVERSATION MANAGERS")
    print("="*70)

    for manager, label in zip(managers, labels):
        stats = manager.get_stats()
        print(f"\n🏷️  {label}:")
        print(f"   Total Turns: {stats['total_turns']}")
        print(f"   Current Messages: {stats['current_messages']}")
        print(f"   Total Words: {stats['total_words']}")
        print(f"   Total Characters: {stats['total_chars']}")
        print(f"   Summaries Created: {stats['summaries_created']}")

        if manager.summary_history:
            print(f"   Latest Summary: {manager.summary_history[-1][:100]}...")

    print("="*70)


compare_managers(
    [manager1, manager2, manager3, manager4],
    ["Basic (turns=5)", "Word limit (50)", "Char limit (200)", "Frequent summarization"]
)


📊 COMPARISON OF CONVERSATION MANAGERS

🏷️  Basic (turns=5):
   Total Turns: 8
   Current Messages: 3
   Total Words: 102
   Total Characters: 728
   Summaries Created: 2
   Latest Summary: This conversation is about Python programming, specifically lists and dictionaries. The user express...

🏷️  Word limit (50):
   Total Turns: 6
   Current Messages: 2
   Total Words: 40
   Total Characters: 325
   Summaries Created: 0

🏷️  Char limit (200):
   Total Turns: 6
   Current Messages: 1
   Total Words: 23
   Total Characters: 144
   Summaries Created: 1
   Latest Summary: This conversation has just begun. The user has greeted with a "Hi there!" and no other topics or inf...

🏷️  Frequent summarization:
   Total Turns: 9
   Current Messages: 3
   Total Words: 143
   Total Characters: 921
   Summaries Created: 3
   Latest Summary: Here is a summary of the conversation:

* The conversation started with a summary of a previous disc...


## Final Summary and Key Takeaways

This notebook demonstrated a comprehensive conversation management system with the following features:

### ✅ Implemented Features:
1. **Conversation History Management**: Maintains messages as list of dictionaries with role and content
2. **Multiple Truncation Options**:
   - By number of turns (keeps last N messages)
   - By word count (keeps messages within word limit)
   - By character count (keeps messages within character limit)
3. **Periodic Summarization**: Automatically summarizes conversation history every K turns using Groq API
4. **Statistics Tracking**: Monitors conversation metrics and summarization activity

### 🔧 Technical Implementation:
- Uses only standard Python libraries plus `requests` and `openai` client
- Groq API integration via OpenAI-compatible interface
- Modular, clean code with comprehensive documentation
- Error handling for API calls

### 📈 Demonstration Results:
- **Demo 1**: Basic truncation by message count with periodic summarization
- **Demo 2**: Word count limitation effectively truncates verbose conversations
- **Demo 3**: Character count limitation provides fine-grained control
- **Demo 4**: Frequent summarization compresses conversation history while preserving context

### 💡 Key Benefits:
- **Memory Efficiency**: Prevents conversation history from growing unbounded
- **Context Preservation**: Summarization maintains important conversation context
- **Flexible Configuration**: Multiple truncation strategies can be combined
- **Production Ready**: Robust error handling and monitoring capabilities